# Expected Value of Cost function in population setting
## Projecting $p$ entries as sigmoids 

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
np.random.seed(seed = 1)

length, n = 500, 3
A, P, Sigma = None, None, None 

if n == 2:
    A = np.array([[0.4, 0.9], [0.1, 0.4]])
    P = np.array([[1.0, 0.0], [0.0, 1.0]])
    Sigma = np.identity(n)
elif n == 3:
    A = np.array([[0.5, 0.0, 0.0], [0.3, 0.4, 0.0], [0.2, 0.3, 0.6]])
    Sigma = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])
    P = np.array([[0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, 0.0]])

As, Ps = A.copy(), P.copy()

print(A)

[[0.5 0.  0. ]
 [0.3 0.4 0. ]
 [0.2 0.3 0.6]]


## Expected Cost

In [4]:
def expected_cost(A, P, As = As, Ps = Ps):
    # base on the distribution of X, no actual data needed.
    # we need the covariance of X_t - X_{t-1}.
    # Then, the expected cost is the trace of this covariance
    P_inv = np.linalg.inv(P)
    Ps_inv = np.linalg.inv(Ps)
    
    B = np.matmul(P_inv, np.matmul(A, P))
    Bs = np.matmul(Ps_inv, np.matmul(As, Ps))
    
    covariance_X = np.matmul(np.linalg.inv(np.identity(n ** 2) - np.kron(Bs, Bs)), Sigma.reshape(n ** 2)).reshape((n, n))
    
    covariance_matrix = Sigma + np.matmul((Bs - B), np.matmul(covariance_X, (Bs - B).transpose()))
    
    return np.trace(covariance_matrix)

## Projecting P onto sigmoid

In [5]:
aparms = int(n * (n + 1) / 2)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def expected_cost_opt(variables):
    # create A
    A = np.zeros((n, n))
    A[np.tril_indices(n)] = variables[:aparms]

    # create P
    P = np.array(sigmoid(variables[aparms:])).reshape(n, n)
    
    ## compute expected cost
    # compute inverses
    P_inv = np.linalg.inv(P)
    Ps_inv = np.linalg.inv(Ps)
    
    # compute B
    B = np.matmul(P_inv, np.matmul(A, P))
    Bs = np.matmul(Ps_inv, np.matmul(As, Ps))
    
    # compute covariance of X
    covariance_X = np.matmul(np.linalg.inv(np.identity(n ** 2) - np.kron(Bs, Bs)), Sigma.reshape(n ** 2)).reshape((n, n))
    
    # compute covariance of X_{val} - X_{pred}
    covariance_matrix = Sigma + np.matmul((Bs - B), np.matmul(covariance_X, (Bs - B).transpose()))
    
    # return cost
    return np.trace(covariance_matrix)

# set dimension
n = 3

# set A
A = np.tril((np.random.rand(n, n)))

# set P
P = np.array([[100, -100, -100], [-100, 100, -100], [-100, -100, 100]])

# define variables
variables = np.append(A[np.tril_indices(n)], P.flatten())

# compute expected cost
expected_cost_opt(variables)

4.759361371323708

## Equality constraints for $n = 3$

In [7]:
def eq_cons_1(variables):
    P = np.array(sigmoid(variables[aparms:])).reshape((n, n))[0]
    return sum(P) - 1

def eq_cons_2(variables):
    P = np.array(sigmoid(variables[aparms:])).reshape((n, n))[1]
    return sum(P) - 1

def eq_cons_3(variables):
    P = np.array(sigmoid(variables[aparms:])).reshape((n, n))[2]
    return sum(P) - 1

def eq_cons_4(variables):
    P = np.array(sigmoid(variables[aparms:])).reshape((n, n))[:, 0]
    return sum(P) - 1

def eq_cons_5(variables):
    P = np.array(sigmoid(variables[aparms:])).reshape((n, n))[:, 1]
    return sum(P) - 1

def eq_cons_6(variables):
    P = np.array(sigmoid(variables[aparms:])).reshape((n, n))[:, 2]
    return sum(P) - 1

cons = [{'type': 'eq', 'fun': eq_cons_1}, {'type': 'eq', 'fun': eq_cons_2}, {'type': 'eq', 'fun': eq_cons_3},
        {'type': 'eq', 'fun': eq_cons_4}, {'type': 'eq', 'fun': eq_cons_5}, {'type': 'eq', 'fun': eq_cons_6}]

## Optimize with many starting points

In [27]:
from scipy import optimize
x0_best = np.random.rand(aparms + n ** 2)
x0 = x0_best

best_results = optimize.minimize(expected_cost_opt, x0, constraints = cons)
results = best_results

attempts, broken_attempts = 0, 0

while (results.fun > 3.05 or results.fun < 0) and attempts < 1000:
    
    attempts += 1
    
    try:
        x0 = np.random.rand(aparms + n ** 2)
        results = optimize.minimize(expected_cost_opt, x0, constraints = cons)
        
        if results.fun < best_results.fun and results.fun > 0:
            x0_best
            best_results = results
        
    except:
        broken_attempts += 1
        print("Error")
    
    print(results.fun)

print(x0_best)
print(best_results.fun)
A_pred = np.zeros((n, n))
A_pred[np.tril_indices(n)] = best_results.x[:aparms]
P_pred = best_results.x[aparms:]

print("Attempts:", attempts)
print("Errored attempts:", broken_attempts)

print(np.round(A_pred, 2))
print(np.round(sigmoid(P_pred).reshape((n, n)), 2))

C:\Users\s165048\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
C:\Users\s165048\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: overflow encountered in matmul


283807017.3573322
5375258052.873922
55280930.60690063
269611.8560598999
106182360.74271417
1350.0708338255702
5652105.616569405
Error
5652105.616569405
150841.27707178076
24562764743101.445
10204.289082349414
10555918805.787647
Error
10555918805.787647
250376108.35571298
3685445.9116508244
2970943.5968343616
70028468922.92181
3454994321.442377
Error
3454994321.442377
Error
3454994321.442377
11567275.397806307
801851615.9984189
876627.3246939245
12404571.220406353
1121094526604.4177
Error
1121094526604.4177
3360976900.4819446
35073140.07573308
547277.4460047943
Error
547277.4460047943
492162796.6558767
Error
492162796.6558767
23739884.761142246
15864264.11661269
1080657.8242224413
3110377563806.73
570756421.6689154
4827927066.702382
Error
4827927066.702382
Error
4827927066.702382
Error
4827927066.702382
6693438.348277304
Error
6693438.348277304
74248.00201523384
4012398.0810699854
832452332.3422098
Error
832452332.3422098
3564874.274939823
1204144.9856524724
1388295695.63965
42259436749

4889739.537337724
209493593.1709629
18874728.35081838
188675234.44859976
398894581.43461096
191758.73150869572
380798913.49129105
Error
380798913.49129105
Error
380798913.49129105
Error
380798913.49129105
14746218531496.727
3053908549344.615
4419066.6353350375
3.640040910561794
1271496.2697310485
1641870.5582494405
315328.29016906745
40.67739266677066
3647126602.3684464
39087.70577992905
Error
39087.70577992905
2537949203.407547
577422109.303593
13938265.601097627
Error
13938265.601097627
10.006345887591308
6.409760532656723e+18
259604378.56642815
3635251.0551888347
6809328.643001007
626552.0169820747
14245575.708323946
120661934.17724241
Error
120661934.17724241
1031022.4537114439
49198227758081.17
903001315443.8127
Error
903001315443.8127
Error
903001315443.8127
55340.909110067325
3322117441.6901135
77361.8588601069
2158.273296932848
485689158.07899594
Error
485689158.07899594
77.08239123372756
595422.936128113
8353619.655796462
Error
8353619.655796462
3895287.957599118
3.00080313853

In [409]:
variables_2 = results.x
results_2 = optimize.minimize(expected_cost_opt, variables_2, constraints = cons)
print(results_2)

     fun: 3.000002428316388
     jac: array([-2.53319740e-06,  1.60336494e-05, -1.47223473e-05, -1.40070915e-05,
       -1.46627426e-05,  1.13248825e-05,  5.48362732e-06,  0.00000000e+00,
       -6.55651093e-07,  0.00000000e+00,  0.00000000e+00, -5.96046448e-08,
        5.96046448e-08,  0.00000000e+00,  0.00000000e+00])
 message: 'Optimization terminated successfully.'
    nfev: 89
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([  0.50313115,   0.30162259,   0.39732725,   0.19890811,
         0.29982063,   0.59934298,  -5.71055772,   4.48970731,
        -4.84587203, -12.7779164 ,  -4.80678081,   4.80642992,
         5.70969995,  -5.80898657,  -8.07466986])
